In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("../out/csvfile/unknown_1-200_ver1.csv")

In [11]:
df = df.sort_values(by="sigma", ascending=False, ignore_index=True)
for i in range(len(df)):
    print(df.loc[i, "smiles"])

O=C(O)C1=C(C(=O)O)C(=CC=C1N(=O)=O)N(=O)=O
O=C(O)C=1C=C(C(=CC1C(=O)O)N(=O)=O)N(=O)=O
O=C(O)C1=C(O)C=C2C(=O)C=3C(O)=C(C(C(=O)C)=C(O)C3C(=O)C2=C1C(=O)O)CC
O=C(O)C1=CC=C(C(=C1C(=O)O)N(=O)=O)N(=O)=O
O=C(O)C1=C(O)C=C2C(=O)C3=C(O)C(O)=C(C(O)=C3C(=O)C2=C1C(=O)O)C4=CC(=CC=C4O)CCN
N#CC=1C=C(C(=O)O)C(=CC1C#N)C(=O)O
O=C(O)C1=CC=C(C(NC(=O)C)=C1C(=O)O)N(=O)=O
O=C(O)C1=C(F)C(F)=C(O)C(F)=C1C(=O)O
O=C(O)C=1C(Br)=CC=C(Br)C1C(=O)O
O=C(O)C=1C(Cl)=CC=C(Cl)C1C(=O)O
O=C(O)C1=C(C(=O)O)C(=CC=C1C)C
O=C(O)C=1C(F)=CC=C(F)C1C(=O)O
O=C(O)C=1C=C(C=C(C(=O)O)C1C(=O)O)N(=O)=O
O=C(O)C=1C(Cl)=CC(Cl)=C(Cl)C1C(=O)O
O=C(O)C=1C(OC)=CC=C(OC)C1C(=O)O
O=C(O)C=1C(F)=CC=C(Br)C1C(=O)O
O=C(O)C=1C(Cl)=CC=C(C1C(=O)O)C(F)(F)F
O=C(O)C=1C=C(C(=O)O)C(C(=O)O)=C(C(=O)O)C1C(=O)O
O=C(O)C=1C(Br)=C(Br)C(Br)=C(Br)C1C(=O)O
O=C(O)C=1C=C(C(=O)O)C(=CC1C(=O)O)S(=O)(=O)O
O=C(O)C1=C(OCC)C=2C=CC=CC2C(OCC)=C1C(=O)O
O=C(O)C1=CC(Cl)=C(C(=O)O)C(C(=O)O)=C1Cl
O=C(O)C1=CC=C(C=C1C(=O)O)S(=O)(=O)N
O=C(O)C=1C(F)=CC=C(C1C(=O)O)C
O=C(O)C=1C(C(=O)O)=C(C(C(=O)O)=C(C

In [12]:
df[["cas", "sigma"]]

,cas,sigma
0,2300-16-5,2.930214
1,90348-28-0,2.768664
2,6219-66-5,2.747609
3,92971-15-8,2.659946
4,14597-16-1,2.515757
...,...,...
171,499793-28-1,1.891871
172,1868072-45-0,1.891701
173,4961-03-9,1.890635
174,835-58-5,1.889630


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

df = pd.read_csv(
    r"C:\Users\kkyom\OneDrive\デスクトップ\pka_activity\ver_3\out\save\value.csv"
)

df = df.dropna(how="any").reset_index(drop=True)

descriptors = [
    "dipole_moment_debye",
    "3and6",
    "lumo_ev",
    "homo_ev",
]


length = len(descriptors)
X = df[descriptors]
y = df["yield"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kernel = (
    ConstantKernel() * RBF([1.0] * length, length_scale_bounds=(1e-2, 1e10))
    + WhiteKernel()
)
model = GaussianProcessRegressor(
    kernel=kernel, n_restarts_optimizer=10, random_state=42
)
loo = LeaveOneOut()
y_true, y_pred = [], []
for i, (train_index, test_index) in enumerate(loo.split(X_scaled)):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)

    y_pred.append(model.predict(X_test)[0])
    y_true.append(y_test.values[0])

    if i < 5:  # 最初の5回だけ表示
        print(f"Iter {i}: Kernel params = {model.kernel_}")
        # もしHOMO（4変数版の4番目）のLength Scaleが小さくなっている回があれば、
        # そのデータにとってはHOMOが「超重要」だということです。

mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

model.fit(X_scaled, y)



Iter 0: Kernel params = 14.9**2 * RBF(length_scale=[11.3, 14.3, 8.89, 7.14e+04]) + WhiteKernel(noise_level=3.52)
Iter 1: Kernel params = 14.7**2 * RBF(length_scale=[11.2, 13.9, 8.84, 4.63e+04]) + WhiteKernel(noise_level=3.44)
Iter 2: Kernel params = 13**2 * RBF(length_scale=[10.9, 11.6, 5.5, 4.64e+04]) + WhiteKernel(noise_level=2.98)
Iter 3: Kernel params = 14.3**2 * RBF(length_scale=[9.7, 15.1, 9.96, 1.48e+04]) + WhiteKernel(noise_level=3.33)
Iter 4: Kernel params = 14.4**2 * RBF(length_scale=[11.4, 13.9, 8.12, 4.1e+04]) + WhiteKernel(noise_level=3.54)


,"kernel kernel: kernel instance, default=NoneThe kernel specifying the covariance function of the GP. If None ispassed, the kernel ``ConstantKernel(1.0, constant_value_bounds=""fixed"")* RBF(1.0, length_scale_bounds=""fixed"")`` is used as default. Note thatthe kernel hyperparameters are optimized during fitting unless thebounds are marked as ""fixed"".",1**2 * RBF(le...noise_level=1)
,"alpha alpha: float or ndarray of shape (n_samples,), default=1e-10Value added to the diagonal of the kernel matrix during fitting.This can prevent a potential numerical issue during fitting, byensuring that the calculated values form a positive definite matrix.It can also be interpreted as the variance of additional Gaussianmeasurement noise on the training observations. Note that this isdifferent from using a `WhiteKernel`. If an array is passed, it musthave the same number of entries as the data used for fitting and isused as datapoint-dependent noise level. Allowing to specify thenoise level directly as a parameter is mainly for convenience andfor consistency with :class:`~sklearn.linear_model.Ridge`.For an example illustrating how the alpha parameter controlsthe noise variance in Gaussian Process Regression, see:ref:`sphx_glr_auto_examples_gaussian_process_plot_gpr_noisy_targets.py`.",1e-10
,"optimizer optimizer: ""fmin_l_bfgs_b"", callable or None, default=""fmin_l_bfgs_b""Can either be one of the internally supported optimizers for optimizingthe kernel's parameters, specified by a string, or an externallydefined optimizer passed as a callable. If a callable is passed, itmust have the signature:: def optimizer(obj_func, initial_theta, bounds): # * 'obj_func': the objective function to be minimized, which # takes the hyperparameters theta as a parameter and an # optional flag eval_gradient, which determines if the # gradient is returned additionally to the function value # * 'initial_theta': the initial value for theta, which can be # used by local optimizers # * 'bounds': the bounds on the values of theta .... # Returned are the best found hyperparameters theta and # the corresponding value of the target function. return theta_opt, func_minPer default, the L-BFGS-B algorithm from `scipy.optimize.minimize`is used. If None is passed, the kernel's parameters are kept fixed.Available internal optimizers are: `{'fmin_l_bfgs_b'}`.",'fmin_l_bfgs_b'
,"n_restarts_optimizer n_restarts_optimizer: int, default=0The number of restarts of the optimizer for finding the kernel'sparameters which maximize the log-marginal likelihood. The first runof the optimizer is performed from the kernel's initial parameters,the remaining ones (if any) from thetas sampled log-uniform randomlyfrom the space of allowed theta-values. If greater than 0, all boundsmust be finite. Note that `n_restarts_optimizer == 0` implies that onerun is performed.",10
,"normalize_y normalize_y: bool, default=FalseWhether or not to normalize the target values `y` by removing the meanand scaling to unit-variance. This is recommended for cases wherezero-mean, unit-variance priors are used. Note that, in thisimplementation, the normalisation is reversed before the GP predictionsare reported... versionchanged:: 0.23",False
,"copy_X_train copy_X_train: bool, default=TrueIf True, a persistent copy of the training data is stored in theobject. Otherwise, just a reference to the training data is stored,which might cause predictions to change if the data is modifiedexternally.",True
,"n_targets n_targets: int, default=NoneThe number of dimensions of the target values. Used to decide the numberof outputs when sampling from the prior distributions (i.e. calling:meth:`sample_y` before :meth:`fit`). This parameter is ignored once:meth:`fit` has been called... versionadded:: 1.3",None
,"random_state random_state: int, RandomState instance or None, default=NoneDetermines random number generation used to initialize the centers.Pass an int for reproducible results across multiple function calls.See :term:`